# NLP y valoracion de Comentarios
Para la evaluación de los diferentes comentarios se hara uso de la libreria *PANDAS* para control de datos, en conjunto con la libreria *Textblob*.
Se tomo como referencia el ejercicio presentado en [[1]](https://www.aprendemachinelearning.com/ejercicio-nlp-cuentos-de-hernan-casciari-python-espanol/), ademas de extraer algunos fragmentos de código alli presentados.

El esquema de trabajo a seguir corresponde a cargar los datos de los comentarios previamente almacenados en un archivo .csv, realizar una transcion de los mismos a un dataframe o arreglo pandas, y finalmente realizar NLP mediante los comandos proporcionados por la libreria *TextBlob*.


In [1]:
#Se importan las librerias
import numpy as np
import pandas as pd
# Librerias Necesarias para trabajo y cadenas de texto
import re
import string


In [2]:
#Lectura del archivo CSV
datos=pd.read_csv('comentarios.csv',sep='\n')
datos.head()#Visualizamos que se hallan cargado correctamente

,"Comentarios extraidos,"
0,"""Claro es lo peor �ltimamente en telefon�a cel..."
1,"""Desde hace un a�o que compr� �ste paquete, ha..."
2,"""Llevo m�s de un a�o con el servicio de claro ..."
3,"""La App de Claro es buena. Lo que no es bueno ..."
4,"""Horrible la se�al. Se me va constantemente el..."


Ahora se realiza la transcion a un Dataframe de pandas, este proceso se reliza para poder aplicar comandos de NLP con mayor facilidad

In [3]:
#Transicion a dataframe
data_df = pd.DataFrame.from_dict(datos)
data_df.columns = ['transcript'] #La columna con informacion se titulará 'transcript'
data_df = data_df.sort_index()#organizacion or indices
data_df.head()

,transcript
0,"""Claro es lo peor �ltimamente en telefon�a cel..."
1,"""Desde hace un a�o que compr� �ste paquete, ha..."
2,"""Llevo m�s de un a�o con el servicio de claro ..."
3,"""La App de Claro es buena. Lo que no es bueno ..."
4,"""Horrible la se�al. Se me va constantemente el..."


Una vez cargados los datos se realiza una limpieza del texto y de caracteres especiales, 

In [4]:
#Funcion para Limpieza de Texto
def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?¿\]\%', ' ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\w*\d\w*', '', text)
    return text


In [5]:
round1 = lambda x: clean_text_round1(x)

In [6]:
data = pd.DataFrame(data_df.transcript.apply(round1))
data.head()

,transcript
0,claro es lo peor �ltimamente en telefon�a cel...
1,desde hace un a�o que compr� �ste paquete ha...
2,llevo m�s de un a�o con el servicio de claro ...
3,la app de claro es buena lo que no es bueno ...
4,horrible la se�al se me va constantemente el...


A continuacion se aplicará TextBlob, la libreria provee el concepto de Polaridad(Parametro entregado entre -1 y 1) usado para evaluar satisfaccion asociadad a sentimientos, y subjetividad(Parametro entregado entre 0 y 1) asociado a que tan diciente u objetivo es un texto.

Ademas es importante mencionar que el analisis de texto funciona unicamente para contenido en ingles, asi que tambien debe aplicarse una traducion, que es provista tambien a traves de la libreria textblob.

In [7]:
#Definiciones lambda para textblob
from textblob import TextBlob
pol = lambda x: TextBlob(x).sentiment.polarity
pol2 = lambda x: x.sentiment.polarity
sub = lambda x: TextBlob(x).sentiment.subjectivity
sub2 = lambda x: x.sentiment.subjectivity
traducir = lambda x: TextBlob(x).translate(to="en")

In [8]:
#Se crea una Nueva Columna que contendra la traduccion del texto
data['blob_en'] = data['transcript'].apply(traducir)
#Se crea una Nueva Columna que contendra el resultado numerico del parametro polaridad
data['polarity'] = data['blob_en'].apply(pol2)
#Se crea una Nueva Columna que contendra el resultado numerico del parametro subjetividad
data['subjectivity'] = data['blob_en'].apply(sub2)

In [9]:
data.head()

,transcript,blob_en,polarity,subjectivity
0,claro es lo peor �ltimamente en telefon�a cel...,"(O, f, , c, o, u, r, s, e, ,, , i, t, , i, ...",-0.227778,0.477778
1,desde hace un a�o que compr� �ste paquete ha...,"(f, o, r, , a, , y, e, a, r, , t, h, a, t, ...",0.038889,0.380556
2,llevo m�s de un a�o con el servicio de claro ...,"(I, , h, a, v, e, , b, e, e, n, , u, s, i, ...",-0.082619,0.589683
3,la app de claro es buena lo que no es bueno ...,"(t, h, e, , c, l, e, a, r, , a, p, p, , i, ...",0.337000,0.522859
4,horrible la se�al se me va constantemente el...,"(h, o, r, r, i, b, l, e, , t, h, e, , s, i, ...",-0.401667,0.616667


Para la evaluacion de la experiencia de usuario, pueden tomarse los valores numericos contenidos en polaridad, subjetividad o ambos. A continuacion se hara uso de la columna polaridad y en base a ella se realiza la asigancacion de una valoracion : *Positiva, Negativa*  ademas de una asignacion aproximada de estrellas de calificacion.
Un ciclo iterativo evalua individualmente los componentes del arreglo y asigna los valores a dos nuevas columnas.

In [10]:
data['Calif']=0
data['Stars']=0

In [11]:
for i in range(len(data)):
    if (data.polarity[i]>0.2):
        data.Calif[i] = 'Positivo'
        data.Stars[i] = 3
        if (data.polarity[i]>0.3):
            data.Stars[i] = 4
        elif (data.polarity[i]>0.4):
            data.Stars[i] = 5
    else:
        data.Calif[i] = 'Negativo'
        data.Stars[i] = 2
        if (data.polarity[i]<0):
            data.Stars[i] = 1    

<ipython-input-11-4722efc15cd7>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.Calif[i] = 'Negativo'
c:\users\asus\miniconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
<ipython-input-11-4722efc15cd7>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.Stars[i] = 2
<ipython-input-11-4722efc15cd7>:13: SettingWithCopyWarning: 
A

In [12]:
data.head()

,transcript,blob_en,polarity,subjectivity,Calif,Stars
0,claro es lo peor �ltimamente en telefon�a cel...,"(O, f, , c, o, u, r, s, e, ,, , i, t, , i, ...",-0.227778,0.477778,Negativo,1
1,desde hace un a�o que compr� �ste paquete ha...,"(f, o, r, , a, , y, e, a, r, , t, h, a, t, ...",0.038889,0.380556,Negativo,2
2,llevo m�s de un a�o con el servicio de claro ...,"(I, , h, a, v, e, , b, e, e, n, , u, s, i, ...",-0.082619,0.589683,Negativo,1
3,la app de claro es buena lo que no es bueno ...,"(t, h, e, , c, l, e, a, r, , a, p, p, , i, ...",0.337000,0.522859,Positivo,4
4,horrible la se�al se me va constantemente el...,"(h, o, r, r, i, b, l, e, , t, h, e, , s, i, ...",-0.401667,0.616667,Negativo,1
